In [ ]:
import sys
sys.path.append('./src')

from utils import *
from osqp_solver import *
from osqp_sqp import *
from osqp_mpc import *
import pinocchio as pin
from pinocchio.visualize import MeshcatVisualizer
import meshcat.geometry as g
import meshcat.transformations as tf
import numpy as np
np.set_printoptions(linewidth=99999999)
    
# Load robot model
urdf_filename = "dependencies/indy-ros2/indy_description/urdf_files/indy7.urdf"
model, visual_model, collision_model = load_robot_model(urdf_filename)

# Setup pinocchio visualizer
viz = MeshcatVisualizer(model, collision_model, visual_model)
viz.initViewer()
viz.loadViewerModel()

# Create MPC solver and optimizer
solver = OSQPSolver(model)
sqp = SQP_OSQP(solver)
controller = MPC_OSQP(model, sqp, solver)


In [ ]:
# initial state - zeros
xstart = np.ones(solver.nx)

# goal ee positions
endpoints = np.array([
    np.array(solver.eepos(np.zeros(6))), 
#    np.array(solver.eepos(-0.8 * np.ones(nq)))
])
print("Endpoints:")
print(endpoints)

# Visualize goals
for i, p in enumerate(endpoints):
    viz.viewer[f'point{i}'].set_object(
        g.Sphere(0.05), 
        g.MeshLambertMaterial(color=0xff0000))

    T = tf.translation_matrix(np.array(p[:]))
    viz.viewer[f'point{i}'].set_transform(T)

In [ ]:
# Run MPC
q_trajectory = controller.run_mpc(xstart, endpoints)

In [ ]:
print_stats(sqp.get_stats())
viz.play(q_trajectory=q_trajectory, dt=0.01)